<a href="https://colab.research.google.com/github/khizar23/Emotion-detection-using-ARTIIFICIAL-INTELLIGENTCE/blob/main/chatbot_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import string
import nltk
import random
from nltk.stem.porter import PorterStemmer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from datetime import datetime, date
import tensorflow as tf
import json

In [14]:
import json

# Opening JSON file
f = open('/content/intent.json',)

# returns JSON object as
# a dictionary
data = json.load(f)

JSONDecodeError: ignored

In [ ]:
stemmer = PorterStemmer()

In [ ]:
stop_words = ['is','am','the','a','an','be','are','were',]


In [ ]:

words = []
y = []
patterns =[]
x = []
classes = []
# getting the classes(tags) output and patterns from 'data'
for intent in data['intents']:
    classes.append(intent['tag'])
    for pattern in intent['patterns']:
# applying tokenizer to convert the sentences into a list of words
        tokens = nltk.word_tokenize(pattern)
# here .extend() is used instead of append() since we don't want to append lists in words
# but its elements i.e words in 'token'
# here 'tokens' is a list of words
        words.extend(tokens)
        patterns.append(pattern)
        y.append(intent['tag'])

# converting to lower case, applyging lemmatization removing the puctuations
# here 'words' is our vocabulary containing all the words
words = [stemmer.stem(word.lower()) for word  in words if word not in string.punctuation and
        word not in stop_words]
# converting the list to set to avoid doubling of words in in 'words'
words = sorted(set(words))
words = list(words)

for list_ in patterns:
    list_ = nltk.word_tokenize(list_)
    list_ = [stemmer.stem(lis.lower()) for lis in list_ if lis not in string.punctuation and lis not in stop_words]
    x.append(list_)


In [ ]:
# applying the one hot encoding on our training data
train_x = []
train_y = []
training = []
empty_list = [0]*len(classes)
for idx,list_ in enumerate(x):
    doc = []
    for word in words:
        doc.append(1) if word in list_ else doc.append(0)
    output = list(empty_list)
    output[classes.index(y[idx])] = 1
    training.append([doc,output])
random.shuffle(training)
training = np.array(training,dtype = object)
train_x = list(training[:,0])
train_x = np.array(train_x)
train_y = np.array(list(training[:,1]))

In [ ]:
ACCURACY_THRESHOLD = 0.99
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):
            print("\nTraining stopped at epoch number:",epoch)
            self.model.stop_training = True
callbacks = myCallback()


In [ ]:
input_shape = (len(train_x[1]),)
output_shape = len(train_y[0])
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
model.fit(x=train_x, y=train_y, epochs=200, verbose=1,callbacks=[callbacks])

2022-12-12 23:07:26.474480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-12 23:07:26.482450: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-12 23:07:26.482543: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (zain-Latitude-E6440): /proc/driver/nvidia/version does not exist
2022-12-12 23:07:26.527039: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
2/2 [==============================] - 2s 5ms/step - loss: 2.8332 - accuracy: 0.0784
Epoch 2/200
2/2 [==============================] - 0s 4ms/step - loss: 2.6795 - accuracy: 0.1765
Epoch 3/200
2/2 [==============================] - 0s 5ms/step - loss: 2.4373 - accuracy: 0.3725
Epoch 4/200
2/2 [==============================] - 0s 5ms/step - loss: 2.2695 - accuracy: 0.3922
Epoch 5/200
2/2 [==============================] - 0s 7ms/step - loss: 2.0134 - accuracy: 0.5490
Epoch 6/200
2/2 [==============================] - 0s 5ms/step - loss: 1.6905 - accuracy: 0.5686
Epoch 7/200
2/2 [==============================] - 0s 4ms/step - loss: 1.4862 - accuracy: 0.6667
Epoch 8/200
2/2 [==============================] - 0s 4ms/step - loss: 1.2008 - accuracy: 0.7843
Epoch 9/200
2/2 [==============================] - 0s 4ms/step - loss: 1.0062 - accuracy: 0.7843
Epoch 10/200
2/2 [==============================] - 0s 4ms/step - loss: 0.7539 - accuracy: 0.8824
Epoch 11/200
2/2 [===========

In [ ]:
from tensorflow import keras


model = keras.models.load_model()
model.save('model.h5')

TypeError: load_model() missing 1 required positional argument: 'filepath'

In [ ]:
# applying bag of words techniques to convert user's text in binary
def bagofwords(msg):
    tokens = nltk.word_tokenize(msg)
    tokens = [stemmer.stem(token.lower()) for token in tokens if token not in string.punctuation and token
              not in stop_words]
    binary_msg = []
    for word in words:
        binary_msg.append(1) if word in tokens else binary_msg.append(0)
    return np.array(binary_msg)


In [ ]:
def prediction(msg):

    message = bagofwords(msg)
    result = model.predict(np.array([message]))
    result = np.argmax(result,axis=1)
    class_index = list(result)[0]
    current_tag = classes[class_index]


    for intent in data['intents']:
        if intent['tag'] == current_tag:
            response = random.choice(intent['responses'])
            return response

In [ ]:
text = input()
response = prediction(text)
while True:
    print(response)
    text = input()
    response = prediction(text)

ANN
1/1 [==============================] - 0s 17ms/step
ANN.txt
Machine learning
1/1 [==============================] - 0s 21ms/step
machine_learning.txt
Tell me about Augmented reality
1/1 [==============================] - 0s 18ms/step
Augmented Reality.txt
augmented reality
1/1 [==============================] - 0s 18ms/step
Augmented Reality.txt
